In [1]:
import torch
from torch import nn
from transformers import AutoTokenizer

In [2]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class IncivilityModule(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, input_ids, attention_mask):
        return torch.sigmoid(self.classifier(self.roberta(input_ids, attention_mask).pooler_output))


## Predictions


In [3]:
trained_model = torch.load("incivility_finetuned_roberta.pt")
trained_model.eval()

trained_model = trained_model.to(device)


In [4]:
def process(tweet):

    encoding = tokenizer.encode_plus(
        tweet,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=False,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )

    return dict(
        input_ids=encoding["input_ids"].flatten(),
        attention_mask=encoding["attention_mask"].flatten()
    )


In [5]:
def make_prediction(tweet):
    item = process(tweet)
    prediction = trained_model(
        item["input_ids"].unsqueeze(dim=0).to(device),
        item["attention_mask"].unsqueeze(dim=0).to(device)
    )
    return torch.round(prediction, decimals=4).flatten().detach().cpu().numpy()


In [6]:
tweet = "You are such a loser! You'll regret everything you've done to me!"

scores = make_prediction(tweet) # The optimal thresholds for measuring Impoliteness and Intolerance were established as [0.7332, 0.4972] for the respective categories of Impoliteness and Intolerance. These thresholds were determined through an analysis of their highest F1-score achieved on a validation set.
scores # [Impoliteness, Intolerance]

array([0.9795, 0.0884], dtype=float32)